In [1]:
#載入
import sqlite3
import pandas as pd
import numpy as np
import datetime
import time
from os import listdir
from os.path import isfile, isdir, join
import os
import talib as tb
import talib.abstract as tbab
import math #判斷nan值用
import csv

conn = sqlite3.connect('TSE.db')
c = conn.cursor()
conn_s = sqlite3.connect('stock_db.db')
c_s = conn_s.cursor()
conn_t = sqlite3.connect('taifex.db')
c_t = conn_t.cursor()
conn_b = sqlite3.connect('bigdeal.db')
c_b = conn_b.cursor()

In [2]:
#篩選function
def fill_nan(df): #填充nan值 大量會無法使用
    col_list = list(df) #欄位名稱
    for col_name in col_list:
        isnull_data = df.isnull()[col_name]
        isnull_index = df[isnull_data].index.tolist() #提取NULL 的Index
        for i in isnull_index:
            while_plus = 1 #同上一筆資料、或上上筆 類推
            while str(df.loc[i][col_name]) == 'nan': #只要是nan 則填如上其數字
                try:
                    df.at[i, col_name] = df.loc[i-while_plus][col_name] #修改df 的值
                except: #第一列沒有填0
                    df.at[i, col_name] = 0
                while_plus = while_plus + 1

    return df
def rsi_back(target_date,input_file): #
    got = []
    for f in input_file:
        data = pd.read_sql_query('SELECT date,收盤指數 FROM "'+ f +'";', conn)
        found = data['date'] == target_date #搜尋第0欄，目標日期資料    
        foundindex = data[found].index.tolist() #data 顯示找到的結果 所在的index 陣列
        if len(foundindex):    
            ok_data = fill_nan(data)
            rsi_val = tb.RSI(ok_data['收盤指數'],timeperiod = 5)
            cal_day = 20 #鎖定20天內的資料
            if ok_data['收盤指數'][foundindex[0]] == ok_data['收盤指數'][foundindex[0]+1-cal_day:foundindex[0]+1].max(): #當天創新高
                ans = rsi_val[foundindex[0]] == rsi_val[foundindex[0]+1-cal_day:foundindex[0]+1].max() #RSI 是否也創新高
                if ans == False:
                    got.append({'stock':f,'signal':'賣出訊號'})
                    #pass
            elif ok_data['收盤指數'][foundindex[0]] == ok_data['收盤指數'][foundindex[0]+1-cal_day:foundindex[0]+1].min(): #當天創新高
                ans = rsi_val[foundindex[0]] == rsi_val[foundindex[0]+1-cal_day:foundindex[0]+1].min() #RSI 是否也創新高
                if ans == False:
                    got.append({'stock':f,'signal':'買進訊號'})
    return got

def avgclose(target_date,input_file):#前三天股價漲跌(包括今天-前天)
    got = []
    for f in input_file:
        data = pd.read_sql_query('SELECT date,收盤指數 FROM "'+ f +'";', conn)
        found = data['date'] == target_date #搜尋第0欄，目標日期資料    
        foundindex = data[found].index.tolist() #data 顯示找到的結果 所在的index 陣列
        try:
            if len(foundindex):  
                ok_data = fill_nan(data)  
                data1 = ok_data['收盤指數'][foundindex[0]-0]
                data5 = ok_data['收盤指數'][foundindex[0]-4]
                got.append({'stock':f,'grow':data1-data5})
        except:
            pass
    return got

def ma5biger(target_date,input_file): #ma5 在上
    got = []
    for f in input_file:
        data = pd.read_sql_query('SELECT date,收盤指數 FROM "'+ f +'";', conn)
        found = data['date'] == target_date #搜尋第0欄，目標日期資料    
        foundindex = data[found].index.tolist() #data 顯示找到的結果 所在的index 陣列
        if len(foundindex):   
            ok_data = fill_nan(data)['收盤指數']
            ma_5 = tb.MA(ok_data,timeperiod = 5)[foundindex[0]]#計算五日MA 並取出當日值
            ma_20 = tb.MA(ok_data,timeperiod = 20)[foundindex[0]] #計算20日MA 並取出當日值
            if ma_5 > ma_20:#如果五日大於20日
                #print(f,ma_5,ma_20)
                got.append({'stock':f,'ma5':'在上'})
        #break
            if ma_5 < ma_20:#如果五日小於20日
                #print(f,ma_5,ma_20)
                got.append({'stock':f,"ma5":'在下'})
    return got

def updown(target_date,input_file): #一個月內是否有震盪
    day = 15#幾天內的資料
    got = []
    for f in input_file:
        data = pd.read_sql_query('SELECT date,收盤指數 FROM "'+ f +'";', conn)
        found = data['date'] == target_date #搜尋第0欄，目標日期資料    
        foundindex = data[found].index.tolist() #data 顯示找到的結果 所在的index 陣列
        if len(foundindex):   
            ok_data = fill_nan(data)['收盤指數']
            ma_5 = tb.MA(ok_data,timeperiod = 5)#計算五日MA 
            ma_20 = tb.MA(ok_data,timeperiod = 20) #計算20日MA
            ma_ary = [] #ma5 與20的差
            down = 0
            grow = 0
            for d in  range(0,day): 
                ma_ary.append(ma_5[foundindex[0]-d]-ma_20[foundindex[0]-d])
            for i in ma_ary: #驗證ma5 是否都在ma20 之上
                if i < 0:
                    down = 1
            for i in ma_ary: #驗證ma5 是否都在ma20 之下
                if i > 0:
                    grow = 1
            if down == 1 and grow == 1: #這個月有之下也有之上的代表震盪
                got.append({'stock':f,"ma":'震盪'})
            elif down == 1 and grow == 0: #這個月有之下也有之上的代表震盪
                got.append({'stock':f,"ma":'跌勢'})
            elif down == 0 and grow == 1: #這個月有之下也有之上的代表震盪
                got.append({'stock':f,"ma":'漲勢'})
                


    return got

def ma_buy(target_date,input_file): #非常不准
    got = []
    cc = 0
    for f in input_file:
        
        data = pd.read_sql_query('SELECT date,收盤指數 FROM "'+ f +'";', conn)
        found = data['date'] == target_date #搜尋第0欄，目標日期資料    
        foundindex = data[found].index.tolist() #data 顯示找到的結果 所在的index 陣列
        if len(foundindex):   
            ok_data = fill_nan(data)['收盤指數']
            ma_5 = tb.MA(ok_data,timeperiod = 5)#計算五日MA 
            ma_10 = tb.MA(ok_data,timeperiod = 10) #計算10日MA
            ma_20 = tb.MA(ok_data,timeperiod = 20) #計算20日MA
            close = pd.read_sql_query('SELECT date,收盤指數 FROM "'+ f +'" WHERE date = '+str(target_date)+';', conn).iloc[0,1]
            got.append({'stock':f,"ma":'未分析'})
            if (ma_5[foundindex[0]] - ma_20[foundindex[0]]) >0: #漲勢
                if close <= ma_5[foundindex[0]]:
                    got[cc]["ma"] = 'ma5買進訊號'
                    
                if close <= ma_10[foundindex[0]]:
                    got[cc]["ma"] = 'ma10買進訊號'
                    
                if close <= ma_20[foundindex[0]]:
                    got[cc]["ma"] = 'ma20買進訊號'
                    pass
            elif (ma_5[foundindex[0]] - ma_20[foundindex[0]]) <0: #跌勢
                if close >= ma_5[foundindex[0]]:
                    got[cc]["ma"] = 'ma5賣出訊號'
                if close >= ma_10[foundindex[0]]:
                    got[cc]["ma"] = 'ma10賣出訊號'
                if close >= ma_20[foundindex[0]]:
                    got[cc]["ma"] = 'ma20賣出訊號'
                    pass
            if got[cc]["ma"] == '未分析':
                got[cc]["ma"] = '無買賣訊號'
                    
        cc += 1 #記數加1 判斷用
    return got

def macdbuy(target_date,input_file):
    got = []
    cc = 0
    for f in input_file:
        
        data = pd.read_sql_query('SELECT date,收盤指數 FROM "'+ f +'";', conn)
        found = data['date'] == target_date #搜尋第0欄，目標日期資料    
        foundindex = data[found].index.tolist() #data 顯示找到的結果 所在的index 陣列
        if len(foundindex):   
            ok_data = fill_nan(data)['收盤指數']
            macd_data = tb.MACD(ok_data)
            osc1 = macd_data[2][foundindex[0]]
            osc2 = macd_data[2][foundindex[0]-1]
            osc3 =  macd_data[2][foundindex[0]-2]
            goin = 0
            if osc2>osc3:#上漲反轉
                if osc1 < osc2: 
                    goin = 1
                    got.append({'stock':f,"macd_osc":'賣出訊號'})
            elif osc2<osc3:#下跌反轉
                if osc1 > osc2: 
                    goin = 1
                    got.append({'stock':f,"macd_osc":'買進訊號'})
            
            if goin == 0:
                got.append({'stock':f,"macd_osc":'無買賣訊號'})
    return got



In [ ]:
#跑全部的日期 買進點
def test(today_str): 
    rsi = rsi_back(today_str,["發行量加權股價指數"])
    avg = avgclose(today_str,["發行量加權股價指數"])
    ma5_ma20 = ma5biger(today_str,["發行量加權股價指數"])
    up_down = updown(today_str,["發行量加權股價指數"])
    ma_signal = ma_buy(today_str,["發行量加權股價指數"])
    macd_signal = macdbuy(today_str,["發行量加權股價指數"])
    got = [today_str]
    if rsi != []: #RSI 有訊號
        got.append('rsi'+rsi[0]['signal'])
        return got #有RSI則不傳回其她結果（可刪除）
    #if avg[0]['grow'] >= 50 or avg[0]['grow']*-1 >= 50: #平盤訊號 (暫時刪除)
    if avg[0]['grow']: #平盤訊號 (暫時刪除)


        if up_down[0]['ma'] != '震盪': #不等於震盪
            if ma_signal[0]['ma'] != "無買賣訊號":
                got.append('均線操作:'+ma_signal[0]['ma'])
        else: #震盪
            if macd_signal[0]['macd_osc'] != "無買賣訊號":
                got.append('MACD操作:'+macd_signal[0]['macd_osc'])
    else:
        pass
    return got

res = []
dddd = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數";', conn)["date"]
for i in range(100,len(dddd)):
    if len(test(dddd[i]))>1:
        res.append(test(dddd[i]))

In [ ]:
#賣出點
def selldot(date,why):
    close_date = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數";', conn)["date"]
    date_f = [] #交易日ARY
    for i in close_date:
        date_f.append(i)
    today_ary_index = date_f.index(date)
    tommorow = date_f[today_ary_index+1]
    t_tommorow = date_f[today_ary_index+2]
    grow = 0
    down = 0
    t_close = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數" where date = '+str(date)+';', conn)["收盤指數"][0]
    tm_close = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數" where date = '+str(tommorow)+';', conn)["收盤指數"][0]
    tmtm_close = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數" where date = '+str(t_tommorow)+';', conn)["收盤指數"][0]

    tse_data = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數";', conn)['收盤指數']
    tse_ma5 = tb.MA(tse_data,timeperiod = 5)
    tse_ma10 = tb.MA(tse_data,timeperiod = 10)
    tse_ma20 = tb.MA(tse_data,timeperiod = 20)
    
    if why == "rsi賣出訊號" or why == "rsi買進訊號":
        for i in close_date:
            date_f.append(i)
        date_r = date_f.index(date)+1
        return date_f[date_r]
    elif why == "MACD操作:賣出訊號" or why == "MACD操作:買進訊號":


        #漲跌趨勢判斷 三天後的股價在MA5哪
        if tmtm_close > tse_ma5[today_ary_index+2]:
            grow = 1
            down = 0
        else:
            down =1
            grow = 0
            
        
        
        if why.find("買")>= 0: #買進
        #隔天賺錢看五日均線賣出
            if tm_close > t_close:
                for dd in range(today_ary_index+1,1000):
                    if grow == 1 and tse_data[dd]<tse_ma5[dd] :
                        return date_f[dd]
                    elif down ==1 and tse_data[dd]>tse_ma5[dd]:
                        return date_f[dd]
                    
        #隔天賠錢 馬上賣掉
            elif tm_close <= t_close:
                return tommorow
            
        elif why.find("賣")>=0:#賣出
        #隔天賠錢 馬上賣掉
            if tm_close >= t_close:
                return tommorow
        #隔天賺錢看五日均線賣出
            elif tm_close < t_close:
                for dd in range(today_ary_index+1,1000):
                    if grow == 1 and tse_data[dd]<tse_ma5[dd] :
                        return date_f[dd]
                    elif down ==1 and tse_data[dd]>tse_ma5[dd]:
                        return date_f[dd]
        
        
        
    
    elif why.find("均線操作:") >= 0:
        if why.find("買")>= 0: #買進
            if tm_close <= t_close:
                return tommorow
            for dd in range(today_ary_index+2,1000):
                if why.find("ma5")>= 0 and tse_data[dd]<tse_ma5[dd]: #買進
                    return date_f[dd]
                elif why.find("ma10")>= 0 and tse_data[dd]<tse_ma10[dd]: #買進
                    return date_f[dd]
                elif why.find("ma20")>= 0 and tse_data[dd]<tse_ma10[dd]: #買進
                    return date_f[dd]
            
        elif why.find("賣")>=0:#賣出
            if tm_close >= t_close:
                return tommorow
            for dd in range(today_ary_index+2,1000):
                if why.find("ma5")>= 0 and tse_data[dd]>tse_ma5[dd]: #買進
                    return date_f[dd]
                elif why.find("ma10")>= 0 and tse_data[dd]>tse_ma10[dd]: #買進
                    return date_f[dd]
                elif why.find("ma20")>= 0 and tse_data[dd]>tse_ma10[dd]: #買進
                    return date_f[dd]

In [ ]:
#模擬買進
money = 200000
target_stock = "00675L"
s_target_stock = "00664R"
for n in range(len(res)):
    i = res[n]
    if len(i)> 1 and i[1].find("均線") == -1 : #如果有買入點 (目前只做RSI)
        date = i[0]
        #確定上一個成交日買點 根本次不同
        close_date = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數";', conn)["date"]
        date_ary = []
        for a in close_date:
            date_ary.append(a)
        try:
            b_date =  date_ary[date_ary.index(date)-1]
            if res[n-1][0] == b_date and i[1] ==res[n-1][1]:
                continue
        except:
            pass
        
        
        try:
            if i[1].find("買") >= 0:
                close = pd.read_sql_query('SELECT date,close FROM "'+ target_stock +'" where date = '+ str(i[0]) +';', conn_s)["close"][0]
                sell_date = selldot(i[0],i[1])
                sell_close = pd.read_sql_query('SELECT date,close FROM "'+ target_stock +'" where date = '+ str(sell_date) +';', conn_s)["close"][0]
                #money = money + ((sell_close - close)*8000)
                dw = int(money/(close*1000))
                money = money + ((sell_close - close)*dw*1000)
                print("買",date,sell_date,(sell_close - close)*8000,sell_close)
            elif i[1].find("賣") >= 0:
                close = pd.read_sql_query('SELECT date,close FROM "'+ s_target_stock +'" where date = '+ str(i[0]) +';', conn_s)["close"][0]
                sell_date = selldot(i[0],i[1])
                sell_close = pd.read_sql_query('SELECT date,close FROM "'+ s_target_stock +'" where date = '+ str(sell_date) +';', conn_s)["close"][0]
                #money = money + ((sell_close - close)*8000)
                dw = int(money/(close*1000))
                money = money + ((sell_close - close)*dw*1000)
                print("賣",date,sell_date,(sell_close - close)*8000,sell_close)
        except:
            pass

print(money)


In [ ]:
#期貨法人
#臺股期貨_自營商',)('臺股期貨_投信',)('臺股期貨_外資及陸資',
#找出
data = pd.read_sql_query('SELECT "date","多空未平倉口數淨額","多空未平倉契約金額淨額(百萬元)" FROM "臺股期貨_投信" ;',conn_t)
data2 = pd.read_sql_query('SELECT "date","多空未平倉口數淨額","多空未平倉契約金額淨額(百萬元)" FROM "臺股期貨_外資及陸資" ;',conn_t)

print(data2)

In [6]:
#判斷當天買進訊號
point = 10977.22 #當天指數

today_str = int(time.strftime("%Y%m%d"))
c.execute('INSERT OR REPLACE INTO "發行量加權股價指數" VALUES('+ str(today_str) +','+str(point)+',null,null,null,null,null,null,null,null,null);')
#today_str = 20190329
close_date = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數";', conn)["date"]
date_f = [] #交易日ARY
for i in close_date:
    date_f.append(i)
yes_index = date_f.index(today_str)-1
yest_int = date_f[yes_index]
def detect(today_str):
    rsi = rsi_back(today_str,["發行量加權股價指數"])
    avg = avgclose(today_str,["發行量加權股價指數"])
    ma5_ma20 = ma5biger(today_str,["發行量加權股價指數"])
    up_down = updown(today_str,["發行量加權股價指數"])
    ma_signal = ma_buy(today_str,["發行量加權股價指數"])
    macd_signal = macdbuy(today_str,["發行量加權股價指數"])
    got = [today_str]
    if rsi != []: #RSI 有訊號
        got.append('rsi'+rsi[0]['signal'])

    #if avg[0]['grow'] >= 50 or avg[0]['grow']*-1 >= 50: #平盤訊號 (暫時刪除)
    if 1 == 1: #平盤訊號 (暫時刪除)
        if up_down[0]['ma'] != '震盪': #不等於震盪
            if ma_signal[0]['ma'] != "無買賣訊號":
                got.append('均線操作:'+ma_signal[0]['ma'])
        else: #震盪
            if macd_signal[0]['macd_osc'] != "無買賣訊號":
                got.append('MACD操作:'+macd_signal[0]['macd_osc'])
    else:
        pass
    print(got)
    
detect(yest_int)
detect(today_str)


[20190419]
[20190422]


In [ ]:
#today1 = 63106         136335845 #期貨口數、淨額
#today1 = 1560258561 #現貨買超
data = pd.read_sql_query('SELECT * FROM "發行量加權股價指數" WHERE "外資及陸資(不含外資自營商)" > 1000000000 and "外資及陸資(不含外資自營商)" < 2000000000;',conn)
data2 = pd.read_sql_query('SELECT * FROM "臺股期貨_外資及陸資" where "多空未平倉口數淨額">60000 and "多空未平倉口數淨額"< 70000;',conn_t)

print(data2['date'])
print(data['date'])
for i in data2['date']:
    for d in data['date']:
        if i == d:
            print(i)

In [ ]:
#data 期貨買超
#data = pd.read_sql_query('SELECT "date","多空未平倉口數淨額","多空未平倉契約金額淨額(百萬元)" FROM "臺股期貨_投信" ;',conn_t)
data2 = pd.read_sql_query('SELECT "date","多空未平倉口數淨額","多空未平倉契約金額淨額(百萬元)" FROM "臺股期貨_外資及陸資" ;',conn_t)['多空未平倉契約金額淨額(百萬元)']
data = pd.read_sql_query('SELECT "收盤指數" FROM "發行量加權股價指數" ;',conn)['收盤指數']
def day_change(data,day): #成長%數
    new_ary = []#

    for i in range(len(data)):
        if i > -1+day:
            new_ary.append((data[i]/data[i-day]-1)*100)
    return new_ary


d = day_change(data2,1) #期貨昨天差
t = day_change(data,1)
#dfab = pd.DataFrame({"A":day_change(data,3),"B":day_change(data2,1)[2:400]})
#print(dfab.A.corr(dfab.B)) #' 相關係數
date = pd.read_sql_query('SELECT "date" FROM "發行量加權股價指數" ;',conn)['date']
data1 = pd.read_sql_query('SELECT "外資及陸資(不含外資自營商)" FROM "發行量加權股價指數" ;',conn)['外資及陸資(不含外資自營商)']
d_c=  day_change(data1,1)
num = 1
for i in day_change(data2,1):
    print(i,date[num],)
    #if 0<i<1:
    #    print(date[num],i,data1[num]/100000000)
    num = num +1

In [66]:
def score(targetdate):
    get_score = 0
    close_date = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數";', conn)["date"]
    date_f = [] #交易日ARY
    for i in close_date:
        date_f.append(i)
    date = str(date_f[date_f.index(targetdate)-1]) +" AND "+str(targetdate)

    data = pd.read_sql_query('SELECT * FROM "發行量加權股價指數" where date BETWEEN '+date+';',conn)
    data1 = pd.read_sql_query('SELECT * FROM "臺股期貨_外資及陸資" where date BETWEEN '+date+';',conn_t)
    data2 = pd.read_sql_query('SELECT * FROM "臺股期貨_投信" where date BETWEEN  '+date+';',conn_t)
    data3 = pd.read_sql_query('SELECT * FROM "臺股期貨_自營商" where date BETWEEN  '+date+';',conn_t)
    data_a = pd.read_sql_query('SELECT * FROM "TX_臺股期貨(TX+MTX/4)_所有" where date BETWEEN  '+date+';',conn_b)
    data_b = pd.read_sql_query('SELECT * FROM "TX_臺股期貨(TX+MTX/4)_近月" where date BETWEEN  '+date+';',conn_b)
    #print(data)
    print_data = [data['自營商(自行買賣)'][1]+data['自營商(避險)'][1],
                  data['投信'][1],
                  data['外資及陸資(不含外資自營商)'][1]+data['外資自營商'][1],
                  data1['多空未平倉口數淨額'][1]-data1['多空未平倉口數淨額'][0],
                  data2['多空未平倉口數淨額'][1]-data2['多空未平倉口數淨額'][0],
                  data3['多空未平倉口數淨額'][1]-data3['多空未平倉口數淨額'][0],
                  data_a['前十大交易人買方'][1]-data_a['前十大交易人買方'][0],
                  data_a['前十大交易人賣方'][1]-data_a['前十大交易人賣方'][0],
                  data_b['前十大交易人買方'][1]-data_b['前十大交易人買方'][0],
                  data_b['前十大交易人賣方'][1]-data_b['前十大交易人賣方'][0]]
    print("現貨自營商:"+str(print_data[0]))
    print("現貨投信:"+str(print_data[1]))
    print("現貨外資:"+str(print_data[2]))
    print("期貨外資:"+str(print_data[3]))
    print("期貨投信:"+str(print_data[4]))
    print("期貨自營商:"+str(print_data[5]))
    print("鉅額所有買方:"+str(print_data[6]))
    print("鉅額所有賣方:"+str(print_data[7]))
    print("鉅額近月買方:"+str(print_data[8]))
    print("鉅額近月賣方:"+str(print_data[9]))
    if print_data[0] >0 and print_data[5]>0:
        get_score += 1
    if print_data[1] >0 and print_data[4]>0:
        get_score += 1
    if print_data[2] >0 and print_data[3]>0:
        get_score += 1
    if print_data[0] <0 and print_data[5]<0:
        get_score -= 1
    if print_data[1] <0 and print_data[4]<0:
        get_score -= 1
    if print_data[2] <0 and print_data[3]<0:
        get_score -= 1
    return get_score
    

close_date = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數";', conn)["date"]
date_f = [] #交易日ARY
for i in close_date:
    pass

In [67]:
for i in range(20180928,20181031):
    try:
        print(score(i))
        print(i)
        print("")
    except:
        pass


現貨自營商:1182746913
現貨投信:178190144
現貨外資:2618866279
期貨外資:-5366
期貨投信:-353
期貨自營商:216
鉅額所有買方:403
鉅額所有賣方:887
鉅額近月買方:50
鉅額近月賣方:177
1
20180928

現貨自營商:444727472
現貨投信:-488881596
現貨外資:2443482586
期貨外資:4387
期貨投信:44
期貨自營商:623
鉅額所有買方:1016
鉅額所有賣方:164
鉅額近月買方:-109
鉅額近月賣方:66
2
20181001

現貨自營商:-1418420940
現貨投信:-63239381
現貨外資:-12237470704
期貨外資:-14613
期貨投信:-478
期貨自營商:-679
鉅額所有買方:-2851
鉅額所有賣方:6966
鉅額近月買方:-2534
鉅額近月賣方:5683
-3
20181002

現貨自營商:-1220782577
現貨投信:-1335349499
現貨外資:323499290
期貨外資:5141
期貨投信:-239
期貨自營商:-2506
鉅額所有買方:-732
鉅額所有賣方:-4600
鉅額近月買方:-104
鉅額近月賣方:-3830
-1
20181003

現貨自營商:-779218514
現貨投信:-813096786
現貨外資:-12971871836
期貨外資:-17668
期貨投信:-387
期貨自營商:3546
鉅額所有買方:1422
鉅額所有賣方:9229
鉅額近月買方:933
鉅額近月賣方:8470
-2
20181004

現貨自營商:-560055371
現貨投信:-1307830173
現貨外資:-6841470886
期貨外資:-1511
期貨投信:-566
期貨自營商:1227
鉅額所有買方:3626
鉅額所有賣方:5245
鉅額近月買方:3606
鉅額近月賣方:5358
-2
20181005

現貨自營商:1038311002
現貨投信:-543464044
現貨外資:-11149313404
期貨外資:-940
期貨投信:792
期貨自營商:493
鉅額所有買方:1088
鉅額所有賣方:649
鉅額近月買方:1008
鉅額近月賣方:1113
0
20181008

現貨自營商:13999253

In [76]:
#個股
def score2(targetdate):
    get_score = 0
    close_date = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數";', conn)["date"]
    date_f = [] #交易日ARY
    for i in close_date:
        date_f.append(i)
    date = str(date_f[date_f.index(targetdate)-1]) +" AND "+str(targetdate)

    data = pd.read_sql_query('SELECT * FROM "3231" where date BETWEEN '+date+';',conn_s)
    data_a = pd.read_sql_query('SELECT * FROM "DX_緯創期貨_所有" where date BETWEEN  '+date+';',conn_b)
    data_b = pd.read_sql_query('SELECT * FROM "DX_緯創期貨_近月" where date BETWEEN  '+date+';',conn_b)
    #print(data)
    print_data = [data['自營商買賣超股數(自行買賣)'][1]+data['自營商買賣超股數(避險)'][1],
                  data['投信買賣超股數'][1],
                  data['外資自營商買賣超股數'][1]+data['外陸資買賣超股數(不含外資自營商)'][1],
                  data_a['前十大交易人買方'][1]-data_a['前十大交易人買方'][0],
                  data_a['前十大交易人賣方'][1]-data_a['前十大交易人賣方'][0],
                  data_b['前十大交易人買方'][1]-data_b['前十大交易人買方'][0],
                  data_b['前十大交易人賣方'][1]-data_b['前十大交易人賣方'][0]]
    print("現貨自營商:"+str(print_data[0]))
    print("現貨投信:"+str(print_data[1]))
    print("現貨外資:"+str(print_data[2]))
    print("鉅額所有買方:"+str(print_data[3]))
    print("鉅額所有賣方:"+str(print_data[4]))
    print("鉅額近月買方:"+str(print_data[5]))
    print("鉅額近月賣方:"+str(print_data[6]))

    

close_date = pd.read_sql_query('SELECT date,收盤指數 FROM "發行量加權股價指數";', conn)["date"]
date_f = [] #交易日ARY
for i in close_date:
    pass
for i in range(20190101,20190413):
    try:
        print(score2(i))
        print(i)
        print("")
    except:
        pass

現貨自營商:-10000
現貨投信:5000
現貨外資:-903075
鉅額所有買方:-3
鉅額所有賣方:-9
鉅額近月買方:-2
鉅額近月賣方:-11
None
20190102

現貨自營商:640000
現貨投信:0
現貨外資:1158000
鉅額所有買方:6
鉅額所有賣方:28
鉅額近月買方:5
鉅額近月賣方:11
None
20190103

現貨自營商:564000
現貨投信:3000
現貨外資:1227363
鉅額所有買方:-7
鉅額所有賣方:-7
鉅額近月買方:-11
鉅額近月賣方:-7
None
20190104

現貨自營商:75846
現貨投信:0
現貨外資:3210000
鉅額所有買方:3
鉅額所有賣方:-3
鉅額近月買方:6
鉅額近月賣方:-4
None
20190107

現貨自營商:-250000
現貨投信:0
現貨外資:-965979
鉅額所有買方:-5
鉅額所有賣方:-3
鉅額近月買方:-6
鉅額近月賣方:-1
None
20190108

現貨自營商:331000
現貨投信:0
現貨外資:4892176
鉅額所有買方:17
鉅額所有賣方:10
鉅額近月買方:-2
鉅額近月賣方:-17
None
20190109

現貨自營商:-288000
現貨投信:0
現貨外資:-768927
鉅額所有買方:0
鉅額所有賣方:-5
鉅額近月買方:-3
鉅額近月賣方:-8
None
20190110

現貨自營商:110000
現貨投信:0
現貨外資:-176793
鉅額所有買方:2
鉅額所有賣方:3
鉅額近月買方:2
鉅額近月賣方:0
None
20190111

現貨自營商:-37000
現貨投信:0
現貨外資:-726640
鉅額所有買方:-12
鉅額所有賣方:-8
鉅額近月買方:-12
鉅額近月賣方:-10
None
20190114

現貨自營商:716000
現貨投信:0
現貨外資:3425084
鉅額所有買方:-7
鉅額所有賣方:3
鉅額近月買方:-19
鉅額近月賣方:-30
None
20190115

現貨自營商:-558000
現貨投信:0
現貨外資:1111978
鉅額所有買方:-56
鉅額所有賣方:-80
鉅額近月買方:-25
鉅額近月賣方:0
None
20190116

現貨自營商:-172000
現貨投信:0
現貨外

In [3]:
print('Hello World', end='\r')
print('Hello World', end='\r')
print('Hello World', end='\r')